In [ ]:
import numpy as np
DIM = (70, 70) # hyperspectral map dimensions

# data

In [ ]:
calibration = np.load(open('data/X_labels.npy', 'rb')) # wavelengths
X = np.load(open('data/X.npy', 'rb')) # measured data, dimensions are (index of measurement, wavelength)

# make hyperspectral map
X.resize(DIM + (X.shape[1],))
X[::2, :] = X[::2, ::-1] # input data has snake index


y = np.random.randint(0, 4, X.shape[:-1]) # dummy clustering model

# layout

In [ ]:
import dash_bootstrap_components as dbc
from dash import html, dcc, no_update, ctx
from jupyter_dash import JupyterDash
from dash import Input, Output

# our modules you can modify
from libs_tools.visualization import plot_spectra, plot_map
import libs_tools.dash.custom_components as cc

In [ ]:
range_slider = plot_spectra([X.mean(axis=(0, 1))], calibration=calibration, colormap=['red'])
range_slider.update_layout(
    yaxis=dict(fixedrange=True,),
    template='plotly_dark',
    plot_bgcolor= 'rgba(0, 0, 0, 0)',
    paper_bgcolor= 'rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, b=0, t=0,),
)
pass

In [ ]:
import plotly.express as px

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY]) # dbc.themes.SLATE, dbc_css
app.title = 'LIBS Segmentation'

app.layout = cc.with_frame([html.Div([
    html.Div([cc.with_frame([
        html.Div(dbc.RadioItems(
            id="class_button",
            className="btn-group",
            inputClassName="btn-check",
            labelClassName="btn btn-outline-primary",
            labelCheckedClassName="active",
            options=[
                {"label": "Option 1", "value": 1},
                {"label": "Option 2", "value": 2},
                {"label": "Option 3", "value": 3},
            ],
            value=1), style={'height': '10vh',}),
        html.Div(cc.with_frame([dcc.Graph(id='x_map')]), style={'height': '90vh',}),
    ])], style={'display': 'flex', 'flex-direction': 'column', 'width': '60vw', 'height': '100vh'}),
    html.Br(),
    html.Div([
        cc.with_frame([dcc.Graph(
            id='range_slider',
            figure=range_slider,
        )]),
            dbc.Row([
                cc.with_frame([dcc.Graph(id='point_plot')]),
            ]),
    ], style={'display': 'flex', 'flex-direction': 'column', 'width': '40vw', 'height': '100vh'}),
    html.Div(id='selected_shapes'),
], style={'display': 'flex'})])

# callbacks

In [ ]:
import json

@app.callback(
    Output('selected_shapes', 'children'),
    Input('selected_shapes', 'children'),
    Input('x_map', 'relayoutData'),
    Input('class_button', 'value'),
)
def update_test(memory, relayout, label):
    if 'shapes' in relayout and ctx.triggered_id != 'class_button':
        memory = json.loads(memory) if memory else []
        memory += [(label, 'path')] # TODO relayout['shapes'][-1]['path']
        memory = json.dumps(memory)
    return memory

@app.callback(
    #[
        Output('x_map', 'figure'),
        #Output('x_map', 'relayoutData'),
    #],
    Input('range_slider', 'relayoutData'),
    Input('selected_shapes', 'children'),
)
def update_X_map(zoom, shapes):
    if zoom is None or "xaxis.autorange" in zoom or 'autosize' in zoom:
        values = X.sum(axis=2)
    else:
        values = X[:, :, (calibration >= float(zoom["xaxis.range[0]"])) & (calibration <= float(zoom["xaxis.range[1]"]))].sum(axis=2)
    values /= values.max()
    fig = plot_map(values=values)
    fig.update_layout(
        template='plotly_dark',
        plot_bgcolor= 'rgba(0, 0, 0, 0)',
        paper_bgcolor= 'rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, b=0, t=0,),
    )
    
    return fig

@app.callback(
    Output('point_plot', 'figure'),
    Input('x_map', 'hoverData'),
)
def update_point_plot(hover):
    if hover is not None:
        x, y = hover['points'][0]['x'], hover['points'][0]['y']
    else:
        x, y = 0, 0
    fig = plot_spectra([X[x, y, :]], calibration=calibration)
    fig.update_layout(
        template='plotly_dark',
        plot_bgcolor= 'rgba(0, 0, 0, 0)',
        paper_bgcolor= 'rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, b=0, t=0,),
    )
    return fig



# run

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=True)